# Экстрактивная суммаризация с BERT

## Описание задачи

Необходимо провести суммаризацию текста, применив методы экстрактивной суммаризации.

Экстрактивная суммаризация – это метод автоматического извлечения ключевых предложений из исходного текста для формирования краткого содержания. В отличие от абстрактивной суммаризации, где создаются новые формулировки, экстрактивный подход берет существующие фразы и упорядочивает их в логичной последовательности.

Что нужно сделать?
Использовать BERT-модели – например, KeyBERT для выделения ключевых фраз, https://human.spbstu.ru/userfiles/files/articles/2024/1/20-35.pdf

Результат:
* код .py, .ipynb
* выводы


## Импорты и настройки

In [1]:
import pandas as pd

In [2]:
# Отображение до 50 столбцов в таблицах
pd.set_option('display.max_columns', 50)

# Сброс кастомного отображения столбцов в таблицах
# pd.reset_option('display.max_columns')

## Обзор данных 

Для данной задачи будем использовать уже распарсенный JSON чата, который был сохранён в CSV-формате

In [3]:
df = pd.read_csv("../def_Klyusnik_A/output/processed_messages_1.csv")

In [4]:
df.head(25)

,name,type,id_first,id,type.1,date,date_unixtime,actor,actor_id,action,inviter,text,text_entities,from,from_id,reply_to_message_id,edited,edited_unixtime,reactions,file,file_name,file_size,thumbnail,thumbnail_file_size,media_type,sticker_emoji,mime_type,duration_seconds,width,height,forwarded_from,saved_from,photo,photo_file_size,via_bot,members
0,💬 Data Practicum Chat,private_supergroup,1379846874,266690,service,2025-02-01T22:40:19,1738438819,Елизавета,user200103497,join_group_by_link,Group,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,💬 Data Practicum Chat,private_supergroup,1379846874,266691,service,2025-02-02T21:59:34,1738522774,Agamet Agametov,user217302209,join_group_by_link,Group,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,💬 Data Practicum Chat,private_supergroup,1379846874,266693,message,2025-02-03T11:28:38,1738571318,NaN,NaN,NaN,NaN,['Всем большой привет! Приглашаю на свой уютны...,"[{'type': 'plain', 'text': 'Всем большой приве...",Olga Varavina,user312724902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,💬 Data Practicum Chat,private_supergroup,1379846874,266694,message,2025-02-03T11:52:20,1738572740,NaN,NaN,NaN,NaN,А у тебя когда будет свой канал про аналитику?,"[{'type': 'plain', 'text': 'А у тебя когда буд...",Илья,user1349934990,266689.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,💬 Data Practicum Chat,private_supergroup,1379846874,266695,message,2025-02-03T11:52:37,1738572757,NaN,NaN,NaN,NaN,Будешь туда голосовухи пятиминутные постить,"[{'type': 'plain', 'text': 'Будешь туда голосо...",Илья,user1349934990,NaN,2025-02-03T11:58:48,1.738573e+09,"[{'type': 'emoji', 'count': 4, 'emoji': '😁', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,💬 Data Practicum Chat,private_supergroup,1379846874,266696,message,2025-02-03T11:55:09,1738572909,NaN,NaN,NaN,NaN,"Потому что сделаны так, будто устарели уже лет...","[{'type': 'plain', 'text': 'Потому что сделаны...",Sergey,user60031833,266654.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,💬 Data Practicum Chat,private_supergroup,1379846874,266697,message,2025-02-03T11:56:57,1738573017,NaN,NaN,NaN,NaN,Подкаст?),"[{'type': 'plain', 'text': 'Подкаст?)'}]",Sergey,user60031833,266695.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,💬 Data Practicum Chat,private_supergroup,1379846874,266698,message,2025-02-03T11:57:17,1738573037,NaN,NaN,NaN,NaN,"Не, это не так раздражает","[{'type': 'plain', 'text': 'Не, это не так раз...",Илья,user1349934990,266697.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,💬 Data Practicum Chat,private_supergroup,1379846874,266699,message,2025-02-03T11:57:36,1738573056,NaN,NaN,NaN,NaN,"Нужны голосовуки с эээ, нууу, и 10 секундными ...","[{'type': 'plain', 'text': 'Нужны голосовуки с...",Илья,user1349934990,NaN,2025-02-03T11:58:05,1.738573e+09,"[{'type': 'emoji', 'count': 1, 'emoji': '😁', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,💬 Data Practicum Chat,private_supergroup,1379846874,266700,message,2025-02-03T11:58:06,1738573086,NaN,NaN,NaN,NaN,"Звонили из ада, просили передать что ждут на м...","[{'type': 'plain', 'text': 'Звонили из ада, пр...",Aleksey Voronin,user234467651,266699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 1007 non-null   object 
 1   type                 1007 non-null   object 
 2   id_first             1007 non-null   int64  
 3   id                   1007 non-null   int64  
 4   type.1               1007 non-null   object 
 5   date                 1007 non-null   object 
 6   date_unixtime        1007 non-null   int64  
 7   actor                83 non-null     object 
 8   actor_id             83 non-null     object 
 9   action               83 non-null     object 
 10  inviter              81 non-null     object 
 11  text                 881 non-null    object 
 12  text_entities        1007 non-null   object 
 13  from                 924 non-null    object 
 14  from_id              924 non-null    object 
 15  reply_to_message_id  689 non-null    f

In [6]:
df.text.head(15)

0                                                   NaN
1                                                   NaN
2     ['Всем большой привет! Приглашаю на свой уютны...
3        А у тебя когда будет свой канал про аналитику?
4           Будешь туда голосовухи пятиминутные постить
5     Потому что сделаны так, будто устарели уже лет...
6                                             Подкаст?)
7                             Не, это не так раздражает
8     Нужны голосовуки с эээ, нууу, и 10 секундными ...
9     Звонили из ада, просили передать что ждут на м...
10                Ты хотел сказать "ждут мастер-класс"?
11    Их сделали, чтобы понравится человеку, котором...
12                      Благодарствую, ещё не проснулся
13    Боже теперь и я в этом рейтинге😆😆😆 и тебе спасибо
14                                          зря конечно
Name: text, dtype: object

In [7]:
df.text_entities.head(15)

0                                                    []
1                                                    []
2     [{'type': 'plain', 'text': 'Всем большой приве...
3     [{'type': 'plain', 'text': 'А у тебя когда буд...
4     [{'type': 'plain', 'text': 'Будешь туда голосо...
5     [{'type': 'plain', 'text': 'Потому что сделаны...
6              [{'type': 'plain', 'text': 'Подкаст?)'}]
7     [{'type': 'plain', 'text': 'Не, это не так раз...
8     [{'type': 'plain', 'text': 'Нужны голосовуки с...
9     [{'type': 'plain', 'text': 'Звонили из ада, пр...
10    [{'type': 'plain', 'text': 'Ты хотел сказать "...
11    [{'type': 'plain', 'text': 'Их сделали, чтобы ...
12    [{'type': 'plain', 'text': 'Благодарствую, ещё...
13    [{'type': 'plain', 'text': 'Боже теперь и я в ...
14           [{'type': 'plain', 'text': 'зря конечно'}]
Name: text_entities, dtype: object

### Вывод

В нашем распоряжении имеется выгрузка чата, которая содержит 1007 строк и 36 столбцов. Имеется множество пропусков в данных.

Ключевые признаки для нас: `text` и `text_entities`

## EDA